# Naive Model & Iterations

**Goal:** predict individual performance on the metrics outlined here: https://www.draftkings.com/help/rules/nba

*Additional Features to consider*
    
    -Lags of same-game metrics currently in data set
    -rest days as a continuous variables

    
 
#### Notes from Jason

*Two main types of games: Tournaments vs cash games*
    
    -cash games head to head
    -Tournaments -- need blow out to win

*Strategies often focus on big names yielding consistent points or finding value on the bench to capitalize on injuries*
    
    Depending on the DK format, (eg tournament vs head-to-head) hidden value needs to also be uncorrelated with other lineups
    

*To find hidden value, focus needs to be on player efficiency and then forecast utilization*
    
    Start by assuming all playing time is equal (ie no garbage time)
        Plot of DKP vs vegas spread suggests there is garbage time --> address later
    Fouls aren't in dataset, but might need to be    

In [29]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn import preprocessing, svm, metrics
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 

In [30]:
df_16 = pd.read_csv('C:\\Users\\dmoxley\\Documents\\GitHub\dk_nba\\dk_2016_working.csv')

### Naive forecast using only variables known prior to start of game

In [31]:
df = df_16
df.dropna(inplace = True) 

In [54]:
# Fit a full model with everything
iv = ['pos_pg','pos_sg','pos_sf',
      #'min_played',
      'actv_flag',
      'start_flag',
      'location',
      'total',
      'spread', #vegas
      'prior_dkp',
      'dkp_diff',
      'three_day_avg',
      'three_day_avg_diff', #Draft king
      'dk_sal',
      'dk_sal_change',
      'rest_none', 'rest_one_day','rest_two_day','rest_over_a_week','rest_three_to_seven_day',
      'prior_game_usage',
      'prior_game_score']

# training and test sets
x = np.array(df[iv])
y = np.array(df['dkp'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3) 

In [55]:
lm = LinearRegression().fit(x_train, y_train) 
y_pred=lm.predict(x_test)
rsq = lm.score(x_test, y_test)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
#mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print('r-squared: '+str(round(rsq,5))
     +'\nMSE: '+str(round(mse,5))
     +'\nRMSE: '+str(round(rmse,5))
     #+'\nMAPE: '+str(round(mape,5))
     ) 
lm_res = pd.DataFrame.from_dict(dict(zip(iv, lm.coef_)), orient='index',columns=['coef'])
lm_res
#lm.intercept_

r-squared: 0.59008
MSE: 82.45523
RMSE: 9.08049


,coef
pos_pg,-7.993135e-02
pos_sg,-4.204656e-01
pos_sf,-5.695059e-01
actv_flag,5.160250e+00
start_flag,3.304440e+00
location,-6.801142e-01
total,8.698936e+00
spread,3.498265e-01
prior_dkp,1.888410e+01
dkp_diff,-2.519136e+00


In [56]:
# normalize features
df_norm = df
df_norm[iv] = df_norm[iv]/df_norm[iv].max()

# training and test sets
x = np.array(df_norm[iv])
y = np.array(df_norm['dkp'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3) 

In [44]:
lm = LinearRegression().fit(x_train, y_train) 
y_pred=lm.predict(x_test)
rsq = lm.score(x_test, y_test)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('r-squared: '+str(round(rsq,5))
     +'\nMSE: '+str(round(mse,5))
     +'\nRMSE: '+str(round(rmse,5))) 
lm_res = pd.DataFrame.from_dict(dict(zip(iv, lm.coef_)), orient='index',columns=['coef'])
lm_res
#lm.intercept_

r-squared: 0.5912
MSE: 81.27728
RMSE: 9.01539


,coef
pos_pg,-1.315945e-01
pos_sg,-3.912849e-01
pos_sf,-3.870477e-01
actv_flag,4.640233e+00
start_flag,3.213910e+00
location,-7.180163e-01
total,7.836819e+00
spread,5.389183e-01
prior_dkp,1.920157e+01
dkp_diff,-2.829055e+00


In [ ]:
# not a lot of features, try Ridge regression
lm = Ridge('lsqr').fit(x_train, y_train) 
y_pred=lm.predict(x_test)
rsq = lm.score(x_test, y_test)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print('r-squared: '+str(round(rsq,5))
     +'\nMSE: '+str(round(mse,5))
     +'\nRMSE: '+str(round(rmse,5))) 
lm_res = pd.DataFrame.from_dict(dict(zip(iv, lm.coef_)), orient='index',columns=['coef'])
lm_res
#lm.intercept_

# Fit random forest

For this pass, 

In [207]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Instantiate model
rf = RandomForestRegressor(n_estimators = 1000 #1000 trees
                           , random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Variables to lag
    '''
         ,'min_played'
         , 'fgm', 'fga', 'pt','rb', 'as', 'st', 'bl', 'to', 'three_point_attempts',
       'three_point_made', 'ftm', 'fta', 'off_reb', 'def_reb', 'fouls',
       'teamfga', 'teamfta', 'teamto', 'teamOR', 'teamDR', 'teamFGM', 'oppfga',
       'oppfta', 'oppto', 'oppOR', 'oppDR', 'oppFGM', 'usage',
       , 'possessions',
       'ppp', 'oppdfeff'
        '''

#### Notes from Jason

*Two main types of games: Tournaments vs cash games*
    
    -cash games head to head
    -Tournaments -- need blow out to win

*Strategies often focus on big names yielding consistent points or finding value on the bench to capitalize on injuries*
    
    Depending on the DK format, (eg tournament vs head-to-head) hidden value needs to also be uncorrelated with other lineups
    

*To find hidden value, focus needs to be on player efficiency and then forecast utilization*
    
    Start by assuming all playing time is equal (ie no garbage time)
        Plot of DKP vs vegas spread suggests there is garbage time --> address later
    Fouls aren't in dataset, but might need to be    
 